# 5章　モデルの評価


## 5.1 モデルの評価とは？
モデルの性能→未知のモデルに対する汎化性能

どのように汎化性能を評価するか？（validation）




## 5.2 バリデーションの手法

### 5.2.1 hold-out法

学習データの一部を学習に使わず、バリデーション用に取っておく事で未知のテストデータに対する予測を模擬する方法。
学習データとテストデータがランダムに分割されていることを前提としている。
ランダムではない（時系列データなど）ときは別の方法を用いる。

以下サンプル実行のための事前準備

In [8]:
! git clone https://github.com/ghmagazine/kagglebook.git

fatal: destination path 'kagglebook' already exists and is not an empty directory.


In [0]:
# ---------------------------------
# データ等の準備
# ----------------------------------
import numpy as np
import pandas as pd

# train_xは学習データ、train_yは目的変数、test_xはテストデータ
# pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）

train = pd.read_csv('/content/kagglebook/input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('/content/kagglebook/input/sample-data/test_preprocessed.csv')

# xgboostによる学習・予測を行うクラス
import xgboost as xgb


class Model:

    def __init__(self, params=None):
        self.model = None
        if params is None:
            self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y, va_x, va_y):
        params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
        params.update(self.params)
        num_round = 10
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label=va_y)
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        self.model = xgb.train(params, dtrain, num_round, evals=watchlist)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred


以下scikit-learnでのデータの分割例

In [10]:
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

# Modelクラスを定義しているものとする
# Modelクラスは、fitで学習し、predictで予測値の確率を出力する

# train_test_split関数を用いてhold-out法で分割する
tr_x, va_x, tr_y, va_y = train_test_split(train_x, train_y,
                                          test_size=0.25, random_state=71, shuffle=True)

# 学習の実行、バリデーションデータの予測値の出力、スコアの計算を行う
model = Model()
model.fit(tr_x, tr_y, va_x, va_y)
va_pred = model.predict(va_x)
score = log_loss(va_y, va_pred)
print(score)

[0]	train-error:0.128533	eval-error:0.1516
[1]	train-error:0.115333	eval-error:0.146
[2]	train-error:0.109333	eval-error:0.1376
[3]	train-error:0.105333	eval-error:0.1364
[4]	train-error:0.096933	eval-error:0.1384
[5]	train-error:0.094667	eval-error:0.1364
[6]	train-error:0.087333	eval-error:0.1296
[7]	train-error:0.084933	eval-error:0.1244
[8]	train-error:0.078133	eval-error:0.1208
[9]	train-error:0.073733	eval-error:0.1172
0.30092523058503867



### 5.2.2 クロスバリデーション

学習データを分割し、hold-out法の手続きを複数回繰り返す方法。
（図５．２）←４分割の例

しばしばCVと略される。

分割されたデータをfold,分割する数をfold数と呼ぶ。

fold数を増やすほど学習データの量を確保できるが、計算時間が増える。（fold数が倍になれば計算する回数は倍になるが、fold数∞の極限で全データなので。。。）
なので、むやみにfold数を増やせば良いと言うことではない。

クロスバリデーションでの評価は、通常各foldの平均を用いる。（データ全体で評価する場合もある）

データ全体でのスコアと、各foldのスコアの平均は評価する指標によって一致したりしなかったりするので注意。
MAEやloglossは一致するが、RMSEは一致しない。
（各foldの平均の方が小さくなる）


In [11]:
# -----------------------------------
# クロスバリデーション
# -----------------------------------
# クロスバリデーションでのデータの分割

from sklearn.model_selection import KFold

# KFoldクラスを用いてクロスバリデーションの分割を行う
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

# -----------------------------------
# クロスバリデーションを行う

from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

# Modelクラスを定義しているものとする
# Modelクラスは、fitで学習し、predictで予測値の確率を出力する

scores = []

# KFoldクラスを用いてクロスバリデーションの分割を行う
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # 学習の実行、バリデーションデータの予測値の出力、スコアの計算を行う
    model = Model()
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    scores.append(score)

# 各foldのスコアの平均をとる
print(np.mean(scores))

[0]	train-error:0.128533	eval-error:0.1516
[1]	train-error:0.115333	eval-error:0.146
[2]	train-error:0.109333	eval-error:0.1376
[3]	train-error:0.105333	eval-error:0.1364
[4]	train-error:0.096933	eval-error:0.1384
[5]	train-error:0.094667	eval-error:0.1364
[6]	train-error:0.087333	eval-error:0.1296
[7]	train-error:0.084933	eval-error:0.1244
[8]	train-error:0.078133	eval-error:0.1208
[9]	train-error:0.073733	eval-error:0.1172
[0]	train-error:0.124	eval-error:0.1512
[1]	train-error:0.1156	eval-error:0.1452
[2]	train-error:0.110933	eval-error:0.1404
[3]	train-error:0.1072	eval-error:0.1396
[4]	train-error:0.097067	eval-error:0.1364
[5]	train-error:0.092133	eval-error:0.1312
[6]	train-error:0.087333	eval-error:0.124
[7]	train-error:0.084133	eval-error:0.1236
[8]	train-error:0.0804	eval-error:0.12
[9]	train-error:0.0768	eval-error:0.1208
[0]	train-error:0.130267	eval-error:0.1576
[1]	train-error:0.1188	eval-error:0.1384
[2]	train-error:0.113067	eval-error:0.1416
[3]	train-error:0.1056	eval-

### 5.2.3 stratified k-fold（層化処理）

分類タスクの場合に、foldごとに含まれるクラスの割合を等しくすること。（stratified sampling)
学習データとテストデータに含まれるデータのクラス毎の割合が同程度であると仮定すると、この手法が妥当でバリデーションの評価が安定する。

ランダムに分割した場合、各クラスの割合にムラが生じて評価のブレが大きくなる可能性があるので、この処理が重要。特に多クラス分類の場合に必要。

クラス数が少ない場合（２クラスなど）は影響が少ないので、行わない場合もある。

以下層化抽出を行うコードの例



In [0]:
# -----------------------------------
# Stratified K-Fold
# -----------------------------------
from sklearn.model_selection import StratifiedKFold

# StratifiedKFoldクラスを用いて層化抽出による分割を行う
kf = StratifiedKFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x, train_y):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]


### group k-fold

何らかのグループ単位で学習データとテストデータが分割されている場合、そのグループを無視してランダムにデータを分割すると性能を過大評価してしまう恐れがある。

（例）顧客データごとに複数の行動履歴があるデータ←顧客の情報をむししてごちゃまぜに分割すると、性能が過大になる恐れ。

このような場合、バリデーションにおいてもグループごとに分割が必要になる。

以下コード例

In [0]:
# -----------------------------------
# GroupKFold
# -----------------------------------
# 4件ずつ同じユーザーがいるデータであったとする
train_x['user_id'] = np.arange(0, len(train_x)) // 4
# -----------------------------------

from sklearn.model_selection import KFold, GroupKFold

# user_id列の顧客IDを単位として分割することにする
user_id = train_x['user_id']
unique_user_ids = user_id.unique()

# KFoldクラスを用いて、顧客ID単位で分割する
scores = []
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_group_idx, va_group_idx in kf.split(unique_user_ids):
    # 顧客IDをtrain/valid（学習に使うデータ、バリデーションデータ）に分割する
    tr_groups, va_groups = unique_user_ids[tr_group_idx], unique_user_ids[va_group_idx]

    # 各レコードの顧客IDがtrain/validのどちらに属しているかによって分割する
    is_tr = user_id.isin(tr_groups)
    is_va = user_id.isin(va_groups)
    tr_x, va_x = train_x[is_tr], train_x[is_va]
    tr_y, va_y = train_y[is_tr], train_y[is_va]

# （参考）GroupKFoldクラスではシャッフルと乱数シードの指定ができないため使いづらい
kf = GroupKFold(n_splits=4)
for tr_idx, va_idx in kf.split(train_x, train_y, user_id):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]


### 5.2.5 leave-one-out(LOO)

学習データのレコードが極めて少ない場合に、fold数＝学習データ（つまりバリデーションデータが１件）とする手法。

n_splitにレコード数を指定すれば良い。（LeaveOneOutクラスもある）

以下LeaveOneOutクラスによるサンプル

In [0]:
# -----------------------------------
# leave-one-out
# -----------------------------------
# データが100件しかないものとする
train_x = train_x.iloc[:100, :].copy()
# -----------------------------------
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
for tr_idx, va_idx in loo.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

GBDTやニューラルネットでLOOを使い、アーリーストッピングを用いると、バリデーションデータに最も都合が良いタイミングで学習が止められてしまうのでモデルの精度が過大評価される。
(LOOでなくてもfold数が多くバリデーションデータが小さい場合同様の問題が起こることがある）

一旦各foldでアーリーストッピングを行った後に、その平均値などで適切なイテレーション数を見積もって再度クロスバリデーションを行うなどして対策する。

## 5.3 時系列データのバリデーション手法

時系列データの場合、過去のデータから時間的に新しいデータに対する予測が求められることが多い。
このため、学習データとテストデータは時系列に沿って分割されることが多く、学習データにはテストデータと同じ期間は含まれない。

単純にランダム分割すると同じ期間のデータが学習できてしまい、時間的に近いデータは似た振る舞いをすることが多いので、モデルの性能を過大評価してしまう危険性が高い。

### 5.3.1 時系列データのhold-out法

（図５．３参照）
学習データのうちテストデータに近い期間をvalidationデータにしてしまう方法。

時間的に近いデータほどデータの傾向も近いという仮定に基づく。
ただし、この仮定は周期性の高いデータなどには成り立たないので、この場合は周期に基づいたvalidationデータを用意したほうが良いかもしれない。

いずれの場合も、テストデータに最も傾向の近いデータを学習に使わないことになるので”もったいない”

そこで、最終的にはバリデーションデータを用いて求めた最適な特徴量やパラメータをそのまま使い、バリデーションデータも含めた再学習を行ったモデルを用いる。
（このモデルの評価はできない）

通常のhold-out法同様バリデーションにデータを有効に使えていないという欠点がある。（バリデーション以外の期間を適切に予測できるモデルになっているかどうかという問題がある）

以下コード例

In [0]:
# ---------------------------------
# データ等の準備
# ----------------------------------
import numpy as np
import pandas as pd

# train_xは学習データ、train_yは目的変数、test_xはテストデータ
# pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）

train = pd.read_csv('/content/kagglebook/input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('/content/kagglebook/input/sample-data/test_preprocessed.csv')

# 時系列データであり、時間に沿って変数periodを設定したとする
train_x['period'] = np.arange(0, len(train_x)) // (len(train_x) // 4)
train_x['period'] = np.clip(train_x['period'], 0, 3)
test_x['period'] = 4

# -----------------------------------
# 時系列データのhold-out法
# -----------------------------------
# 変数periodを基準に分割することにする（0から3までが学習データ、4がテストデータとする）
# ここでは、学習データのうち、変数periodが3のデータをバリデーションデータとし、0から2までのデータを学習に用いる
is_tr = train_x['period'] < 3
is_va = train_x['period'] == 3
tr_x, va_x = train_x[is_tr], train_x[is_va]
tr_y, va_y = train_y[is_tr], train_y[is_va]


### 5.3.2 時系列データのクロスバリデーション（時系列に沿って行う方法）

（図５．４参照）

クロスバリデーションと似ているが、時間的な順序に注意を払って過去データで学習→未来のデータでvalidationという形になるようにして学習させる。

テストデータでの予測モデルも過去データで学習して未来のデータを予測するのでその状況を再現するようにして検証する。

最も古いデータから学習を行うか、図５．５の右側のように、学習データの期間の長さを揃えることもできる。
前者の場合は各foldごとにデータの量が異なるので注意が必要。

あまり古いデータは性質が違って参考にならない場合などもあるので、期間やfold数などはデータの性質や計算負荷などによって考えて決めることになる。

以下コード例。



In [0]:
# -----------------------------------
# 時系列データのクロスバリデーション（時系列に沿って行う方法）
# -----------------------------------
# 変数periodを基準に分割することにする（0から3までが学習データ、4がテストデータとする）
# 変数periodが1, 2, 3のデータをそれぞれバリデーションデータとし、それ以前のデータを学習に使う

va_period_list = [1, 2, 3]
for va_period in va_period_list:
    is_tr = train_x['period'] < va_period
    is_va = train_x['period'] == va_period
    tr_x, va_x = train_x[is_tr], train_x[is_va]
    tr_y, va_y = train_y[is_tr], train_y[is_va]

# （参考）TimeSeriesSplitの場合、データの並び順しか使えないため使いづらい
from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits=4)
for tr_idx, va_idx in tss.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

### 5.3.3 時系列データのクロスバリデーション（単純時間分割）

データ同士の時間的な前後関係をあまり気にしなくて良い場合もある。そのような場合バリデーションデータより将来のデータを学習に含めても問題ない。

→図５．６のように単純に時間分割してクロスバリデーションする。

In [0]:
# -----------------------------------
# 時系列データのクロスバリデーション（単純に時間で分割する方法）
# -----------------------------------
# 変数periodを基準に分割することにする（0から3までが学習データ、4がテストデータとする）
# 変数periodが0, 1, 2, 3のデータをそれぞれバリデーションデータとし、それ以外の学習データを学習に使う

va_period_list = [0, 1, 2, 3]
for va_period in va_period_list:
    is_tr = train_x['period'] != va_period
    is_va = train_x['period'] == va_period
    tr_x, va_x = train_x[is_tr], train_x[is_va]
    tr_y, va_y = train_y[is_tr], train_y[is_va]


### 5.3.4 時系列データのバリデーションの注意点

時系列データでは、タスクの設計やデータの性質や分割のされ方で行うべきバリデーションが変わる。

タスクに応じて特殊なバリデーションを行うのが有効な場合もある。

特徴量の作成でも注意が必要で、テストデータに対して利用できる情報が何かを意識しないと危険。



### author's opinion

時系列にそって行う方法が安全なのでこちらが基本だが、目的変数が過去の目的変数の情報をそれほど持たない場合や、使えるデータが少なく十分なデータで学習できない場合には、単純に時間で分割する方法が有効な場合もある。

#### 大まかな方針

データが十分にあるバア愛は、時系列に沿った方法でバリデーションを行う。
バリデーションの区切りはデータの時間的な粒度を見て決める。区切る期間が大きすぎると、スコアのブレがデータなのか学習データの機関の違いなのかを考察しづらくなる。

予測精度が安定しているかを確認したり、バリデーションスコアとLeaderBoardの相関を見ながら、バリデーション期間と学習データの期間を考える。
あえて同じバリデーションデータに対して使う学習データの期間をずらしてみて、どの程度予測スコアが変動するかを見るのも良い。

データが十分にないときは、単純に時間で分割する方法のバリデーションとしたり、データについてのドメイン知識から仮設を立ててうまく特徴量を作ることなどが考えられる。

### 5.3.5 Recruit Restaurant Visitor Forecasting

飲食店の将来の来客数を予測するタスクの例。

筆者は、学習データの末端４週間のうち、予測日の曜日に一致する日のみをバリデーションに用いた。
> 曜日を合わせることと、同じ曜日の複数日をバリデーションデータに加えることで、日による評価のばらつきを低減するのが狙い

時間的な傾向が大きいデータでは、あまり過去のデータをバリデーション対象に含めると、テストデータに対する汎化性能と乖離する恐れがある。
→試行錯誤して決める


最終的にバリデーションデータも含めた学習を行いモデルを作成し直す。（モデルのパラメータはバリデーションで最適化されたものを用いているので問題ないと判断）




### 5.3.6 Santander product Recommendation

Santander銀行の顧客ごとの金融商品の購入商品を予測するタスク。

学習データが２０１５年２月〜２０１６年５月
予測対象が２０１６年６月
←前月までのデータを学習に使える

このような場合、基本的な戦略は４月までを学習データに５月をバリデーションデータにする。

#### 筆者の戦略

このコンペは月ごとのデータが大きく、単月データでも十分な性能が出るので、２０１６年の４月、３月と一月分のデータで複数のモデルを作成しそれらのアンサンブルを作成。
この際のバリデーションデータに２０１６年５月のデータを用いた。

２０１６年の５月のデータを学習に使っていないので、さらに５月のデータでモデルを作成し４月のデータをバリデーションデータとしてアンサンブルに加えた。
（リークが発生しないようにちゅうしないといけない）

このコンペは商品によって年間の周期性が強く現れるものがあるのが特徴。
これを予測するのには学習データやバリデーションデータに直近のデータを用いるのは適切ではない。
そこで１年前の２０１５年６月のデータを学習データに用いる事が考えられるが、バリデーションデータがない。
そこで、（苦肉の策として）２０１５年６月のデータを学習データとし、２０１５年７月のデータをバリデーションデータとした。
テストデータに対する制度を正しく見積もることはできないが、特徴量選択やパラメータチューニングの指針にはできると判断した。（さらにPublic Leaderboardのスコアも参照して確認）


## 5.4 バリデーションのポイントとテクニンク

## 5.4.1 バリデーションを行う目的

- モデル改善の指針となるスコアを出す

正しくバリデーションできていることは、モデルの性能評価に重要。
コンペでの評価指標とあえて異なる（安定した）評価指標を行うという方法もある。

- テストデータに対するスコアやそのばらつきを見る

バリデーションのスコアから、分析コンペの評価指標でのスコアを見積もる。見積もったスコアとpublic leaderboardのスコアを比較して考察する。


### 5.4.2 学習データとテストデータの分割を真似る

どのように分割すればよいか迷ってしまった場合に、学習データとテストデータの分割の仕方を真似してしまう方法がある。

テストデータの予測に使えるのと同等の情報をつかって、バリデーションデータの予測をしたことになり、
テストデータの予測をするモデルの適切な評価と言える。

データの分割が複雑すぎる場合は真似できない場合もあるが、できるだけ分割の方法を近づけるバリデーションを考えることはできる。

（例）図５．７／５．８
（１）のケース　：　ランダムに抽出された半分のユーザーがテストデータで残り半分が学習データ
（２）のケース　：　その時点でのすべてのユーザーがテストデータ。過去の各月松のユーザーと翌月に解約したかどうかの情報が与えられる。

（１）通常のクロスバリデーションで良い
データがランダムかどうかは留意する必要がある。例えば学習データとテストデータが地域で分割されていると、バリデーションでも地域でgroup K-foldを行うほうが良い。

（２）時系列に沿って行うバリデーションを行う
この場合は、ランダムにバリデーションを行ってしまうと、時間的な順序が逆転したり同じ月のデータを使ってしまったりしてフェアでない評価になってしまう。



### 5.4.3 学習データとテストデータの分布が違う場合

時間や地域などをもとに学習データとテストデータを分割した場合、それぞれなにかのパラメータが違う分布からのサンプルとして与えられたと考えられる場合がある。

時系列データで時間に沿って分割した場合などがこれに該当する。

このような場合は、学習データと同じ分布のデータを予測するだけでは不十分で、テストデータの分布のデータで良い予測をしなければならない。

分布の違いに頑強なモデルを作るのが理想だが、Leaderboardのスコアを参考にテストデータに合わせていくことも必要かもしれない。

対応策

- 学習データとテストデータの傾向の違いを、データの作成過程や探索的データ分析（EDA)をもとに考察する
- adversarial validation の結果や public leaderboardのスコアを参考に、Leaderboardのスコアに相関するバリデーション方法を確立する
- モデルを複雑にしすぎない。効く理由が説明できる特徴量を使うことで、分布の違いに頑強なモデルを作る。
- さまざまなモデルの平均を取るアンサンブルによって予測を安定させる。
- adversarial validationのスコアが低くなるように特徴量を変換し、分布の違いに影響され辛くする。

（EDAについてはテキスト１章参照）
大雑把に言うと、（色々分析←平均出したりグラフ書いたり。。して）データについて理解すること

#### adversarial validation

手法
- 学習データとテストデータを統合し、テストデータかどうかを目的変数とするモデルを作成する。
- モデルに基づいて、それぞれのレコードがテストデータである確率の予測値を出力する
- テストデータである確率が高いと予測される学習データを一定数選んでバリデーションデータとする

学習データとテストデータを統合し、テストデータかどうかを分類するモデルを作成
分類精度が０．５を十分上回る（つまりテストデータと学習データが異なるため予測可能性がある）場合に使う。

（AUCは７５ｐ参照。完全予測で１．０、完全ランダムで０．５）

#### Author's opinion

学習データとテストデータの差異の要因が分かる場合は、それを模倣することでより確実なバリデーションを構築できる。
EDAで違いが発見できなくても、adversarial validationで判別に効いている特徴量を確認し、色々仮設を立てて再度EDAを行ってみると良い。

リークなどの問題で、学習データとテストデータで異なる性質を持つ特徴量を作ってしまった場合、与えられたデータの違いを考察するという目的にこの特徴量は使わないほうが良い。

手元のバリデーションとLeaderboardのスコアの整合が取れないときは、adversarial validationを用いて自身の作成した特徴量のどこに問題があるかを調べるのも有効。



### 5.4.4 Leaderboardの情報の活用

"trust your CV" : Leaderboardのスコアに惑わされずに、バリデーションによってモデルを適切に評価することが重要。
ただし、Leaderboardのスコアはうまく参考にできることもある。

#### バリデーションとの差異を考察する

バリデーションとLeaderboardのスコアの水準と動きが整合していない場合、

- 偶然
- バリデーションデータとテストデータの分布が異なる
- バリデーションの設計が不適切

> 偶然の場合は、Publicのテストデータと同じデータ数のhold-outのデータ（複数）のスコアのばらつきと比べることで、偶然の範囲かどうかを評価できる。

> データの分布が異なる場合は、データの性質や分割の方法について考察する。例えば、時系列データを時間によって分割していると分布が異なる可能性が高い。月ごとにバリデーションを行うことで月ごとのスコアのブレを確認できる。このようなケースではDiscussionでバリデーションのスコアとLeaderboardのスコアの乖離について議論するトピックが立つことが多い。ない場合はバリデーションが不適切な可能性を疑ったほうが良い

#### データの分割とLeaderboardスコア

図５．９参照

1の場合


> データがランダムに分割。この場合はpublicリーダーボードは気にせずクロスバリデーションで評価すれば良い。

2の場合

> 訓練データが少ない場合。public Leaderboardのスコアも利用することでより安定した評価が期待できる

3の場合

> publicのデータのほうがprivateのテストデータに近いのでpublicLBのスコアが良ければPrivateのテストデータも良い可能性が高い。（が、過剰適合しているだけの可能性もあるので注意）

4の場合

> PublicとPrivateのテストデータが同じ期間でランダム分割されている。このケースではpublicがよければprivateも良い可能性が高いので、PublicLBの利用が強く推奨される。

#### shake up

public LBに過適合してしまっていたり、public LBのテストデータのレコードが少なすぎたりすると、PublicのスコアがよくてもPrivateで一気に順位が入れ替わることがあり、これをshake upという。



### 5.4.5 バリデーションやpublic LBへの過適合

バリデーションデータやPublicLBを参照して、パラメータのチューニングを繰り返しすぎると、
バリデーションデータやPublic LBに過適合してしまい、実力以上のスコアが出ることがある。

各提出におけるバリデーションのスコアと、public LBのスコアをプロットして感覚的にブレの影響を掴んだり、クロスバリデーションの分割を変える事が考えられる。

#### クロスバリデーションの分割を変える

パラメータチューニングのためのバリデーションの分割と、
モデルの評価のためのバリデーションの分割を変えるという方法が考えられる。

分割を変えることは、乱数のseedを変えれば良い。
データの全体が変わっていないが、public leaderboradのデータをバリデーションに用いていないhold-out相当なので、この方法で十分な場合が多い。

より保守的には、hold-outデータを取り分けておき、残りのデータでクロスバリデーションによりパラメータチューニングを行い、hold-outデータで評価を行う方法もある。
データが少なくなるので分析コンペではあまり使われない。



### 5.4.6 クロスバリデーションのfoldごとに特徴量を作り直す

#### 時系列データでない場合

foldごとに特徴量を作り直す典型的な例は、target Encoding（データ全体の平均を利用するので）
学習データのencodingを行うときにはテストデータの目的変数は使えないが、この制約をバリデーションにおいても再現するために、foldごとにその時の学習データの目的変数のみを使ってtarget encodingし直す必要がある。

#### 時系列データの場合

図５．１１のように
テストデータとバリデーションデータで使うデータの期間を整合的にすることで、評価の整合性を保つ。

図の右側のように評価対象より先の時点のログデータを使って特徴量を作ってはいけない。


### 5.4.7 使える学習データを増やす

与えられたデータから新たなデータを生成して学習データを増やすことができる。
（data argumentation)
画像データを扱うタスクで、画像を回転・反転させたり歪ませることで学習データを増やすことはよく行われている。

テーブルデータではこのようなことは難しいが可能な例もある

#### instacart market basket analysis

オンラインの食料品配達サービスで、前回に引き続き注文される商品を予測するタスク。

各ユーザーの最新の注文のみがtrainとtestに分割され、それ以外はprior扱い。
priorを学習データとして使うことができる。
（これって水増しの例になっている。。？？？？）


#### Recruit restaurant visitor forecasting

ランダムに一分のデータを削ったあとに特徴量作成を行うことで、新たに学習データを作り学習データを増やす方法が使われた。

各店舗の最初のデータは開店日ではなく、サービスへの登録日であると予測できた。
この場合各店舗のサービス登録日が異なった場合のデータを、図５．１３のように各店舗の先頭のデータを削ることで作ることができる。

このような処理の後にtarget encodingなどの特徴量作成を行うと、データを削らない場合と比べて特徴量が少し異なる学習データを新たに生成できる。

この例では、この増やした特徴量を元のデータに加えるのではなく、別のモデルの学習に使い、モデルの予測のアンサンブルを用いていた。

このプロセスはテストデータに対しても行われていた。(test time argumentationと類似の手法）
